In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
from config import params

#prepairing connection to database
engine = create_engine(params).connect()
#allows querying database(pulling data)
session = Session(engine)
#reflect the tables in the database
Base = automap_base()
Base.prepare(engine, reflect=True)
#mapped classes are created with default matchings of the tables' names
# extracting data from three tables below
jobs = Base.classes.jobs
salaries = Base.classes.salaries
skills = Base.classes.skills
#queries that request all the attributes contained in each table
result_jobs = session.query(jobs)
result_salaries = session.query(salaries)
result_skills = session.query(skills)
#converting each queried table into dataframe
jobs_df = pd.read_sql(result_jobs.statement, engine)
sal_df = pd.read_sql(result_salaries.statement, engine)
skills_df = pd.read_sql(result_skills.statement, engine)



/var/folders/kc/j4ss26096jv9xkwkcr7z36kr0000gn/T/ipykernel_88035/1780175812.py:13: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [2]:
import functools as ft
# create a list of three dataframes
dfs = [jobs_df, sal_df, skills_df]
# with reduce () function merge dataframes into one
#lambda function merges 2 elements first, then the resulting df with the remaining one
joined = ft.reduce(lambda left,right:pd.merge(left,right,on=['id'], how='outer'),dfs)

In [3]:
joined

,id,title,company_name,location,via,extensions,posted_at,schedule_type,work_from_home,date_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,Data Analyst Technical - Consultant,Intermountain Healthcare,"Oklahoma City, OK",via Monster,"['19 hours ago', 'Full-time', 'Health insurance']",19 hours ago,Full-time,None,00:10.1,None,None,NaN,NaN,NaN,NaN,NaN,NaN,"['vba', 'spreadsheet', 'javascript', 'word', '..."
1,1,"Sr Analyst, Data Engineer",Estee Lauder Companies,Oklahoma,via ZipRecruiter,"['12 hours ago', 'Full-time']",12 hours ago,Full-time,None,00:10.1,None,None,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,Legal Data Analyst,DISYS,United States,via Indeed,"['4 hours ago', '21 to 23 an hour', 'Contractor']",4 hours ago,Contractor,None,00:10.1,21 to 23,an hour,22.0,21.0,23.0,22.0,NaN,45760.0,[]
3,3,Data analyst,Modis,United States,via BeBee,"['13 hours ago', 'No degree mentioned']",13 hours ago,None,None,00:10.1,None,None,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,4,Data Analyst (Risk Adjustment Consulting Resea...,Cambia Health Solutions,United States,via LinkedIn,"['23 hours ago', 'Full-time', 'Health insuranc...",23 hours ago,Full-time,None,00:10.1,None,None,NaN,NaN,NaN,NaN,NaN,NaN,"['sas', 'matlab', 'sql', 'spss', 'r']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11201,11201,Senior Data Analyst,Oracle,"Jefferson City, MO",via Trabajo.org,"['19 hours ago', 'Full-time', 'Health insuranc...",19 hours ago,Full-time,None,00:11.6,None,None,NaN,NaN,NaN,NaN,NaN,NaN,"['go', 'tableau', 't-sql', 'spreadsheet', 'sha..."
11202,11202,Principal Data Analyst,Fidelity Investments,"Tishomingo, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,None,00:11.6,None,None,NaN,NaN,NaN,NaN,NaN,NaN,"['sap', 'snowflake', 'sql', 'hadoop', 'aws']"
11203,11203,Senior Data Analyst,Oracle,"Kansas City, MO",via Trabajo.org,"['13 hours ago', 'Full-time', 'Health insuranc...",13 hours ago,Full-time,None,00:11.6,None,None,NaN,NaN,NaN,NaN,NaN,NaN,"['go', 'sql']"
11204,11204,Principal Data Analyst,Fidelity Investments,"Marietta, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,None,00:11.6,None,None,NaN,NaN,NaN,NaN,NaN,NaN,"['sap', 'snowflake', 'sql', 'hadoop', 'aws']"


In [4]:
# saving joined dataframe(jobs, skills, salary) into data folder as csv file
joined.to_csv('data/jobs.csv', index=False)